In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np
import re

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

import data_tools as dt

from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from IPython.core.display import HTML
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

rng = np.random.default_rng(12898)

SUPPORT_TAGS = ['DEFINITELY SUPPORT', 'SOMEWHAT SUPPORT']
OPPOSE_TAGS = ['DEFINITELY OPPOSE', 'SOMEWHAT OPPOSE']
ORDERED_TAGS = ['DEFINITELY SUPPORT', 'SOMEWHAT SUPPORT', 'SOMEWHAT OPPOSE', 'DEFINITELY OPPOSE']

APPROVE_TAGS = ['APPROVED', 'APPROVED IN PART OR WITH MODIFICATIONS']

In [2]:
dfa = dt.get_agenda_items(verbose=False)
dfm = dt.get_minutes(verbose=False)
dfl = dt.get_supplemental_docs(verbose=False)

In [7]:
# Summary statistics on voting patterns
total_ayes = dfm['n_ayes'].sum()
total_nays = dfm['n_nays'].sum()
total_abstains = dfm['n_abstained'].sum()
total_absences = dfm['n_absent'].sum()
total_recusals = dfm['n_recused'].sum()
n_unanimous = (dfm['n_nays']==0).sum()
n_contested = (dfm['n_nays']>0).sum()

print(f"Number of ayes:        {total_ayes:,.0f}")
print(f"Number of nays:        {total_nays:,.0f}")
print(f"Number of abstentions: {total_abstains:,.0f}")
print(f"Number of absences:    {total_absences:,.0f}")
print(f"Number of recusals:    {total_recusals:,.0f}")
print("")
print(f"Number of unanimous votes:   {n_unanimous:,.0f}")
print(f"Number of contested votes:   {n_contested:,.0f}")

Number of ayes:        4,838
Number of nays:        91
Number of abstentions: 4
Number of absences:    1,245
Number of recusals:    55

Number of unanimous votes:   664
Number of contested votes:   66


In [8]:
dfm['ayes_count']

0      0
1      0
2      0
3      0
4      0
      ..
725    0
726    0
727    0
728    0
729    0
Name: ayes_count, Length: 730, dtype: int64